# Developing the Basic LLM Chatbot with Ollama

This notebook documents the development of a basic LLM-powered chatbot using LangChain, Ollama, PostgreSQL, and Streamlit.

## 1. Choice of Offline LLM API Platform and Model

The project was developed in a **rapid prototyping approach**, so we prioritized ease of setup and integration. For this reason, we chose **Ollama** as the offline LLM API platform. The following factors were considered why we chose Ollama for the chabot development:
- Easy local setup (tutorial: [https://ollama.com/download/linux](https://ollama.com/download/linux))
- Docker support (see: [https://ollama.com/blog/ollama-is-now-available-as-an-official-docker-image](https://ollama.com/blog/ollama-is-now-available-as-an-official-docker-image))
- Streamlined model serving with REST endpoints
- Compatible with LangChain

For the model, we chose **Llama 3.2** mainly for its 1) exceptional performance and efficiency, and 2) due to hardware constraints. Its lightweight nature also allows for more portability when deploying the chatbot on different machines.

The model was pulled using the following command:
```
ollama pull llama3.2
```


## 2. Setting Up PostgreSQL with Docker

We use Docker Compose to spin up a PostgreSQL instance.

**`docker-compose.yml` excerpt:**

In [ ]:
# NOTE: Do not run this cell! It is only for documentation purposes.

services:
  db:
    image: postgres:14
    environment:
      POSTGRES_USER: postgres
      POSTGRES_PASSWORD: password
      POSTGRES_DB: chat-history
    ports:
      - "5432:5432"

We can also start a PostgreSQL instance independently in Docker by running the following command:

In [ ]:
# NOTE: Do not run this cell! It is only for documentation purposes.

docker run -d --name postgres-db -e POSTGRES_DB=chat-history -e POSTGRES_USER=postgres -e POSTGRES_PASSWORD=password -p 5432:5432 postgres:14

## 3. Backend: LangChain + langchain-postgres

The backend is built using LangChain. 

We started the backend by using the `langchain-ollama` to connect to the Ollama model and get a response. A sample code is shown below:

In [ ]:
# NOTE: The variables in this notebook are based on the active session I have in my local machine.
# You may need to change the values if you want to run the code.

from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv

load_dotenv()

OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")

llm = ChatOllama(model=OLLAMA_MODEL, base_url=OLLAMA_URL)

response = llm.invoke("Hello, how are you?")

print(response.content)

I'm just a language model, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?


After this, we started building the database connection using `langchain-postgres` to store the chat history. The following code snippet shows how we use `PostgresChatMessageHistory` to persist chat sessions in the database.

For creating tables in the database, we can use the following code:

In [18]:
# NOTE: The variables in this notebook are based on the active session I have in my local machine.
# You may need to change the values if you want to run the code.

from langchain_postgres import PostgresChatMessageHistory
import psycopg
import os
from dotenv import load_dotenv

load_dotenv()

CHAT_HISTORY_TABLE = os.getenv("CHAT_HISTORY_TABLE", "chat_history")

def get_connection():
    return psycopg.connect(
        host=os.getenv("POSTGRES_HOST", "localhost"),
        port=os.getenv("POSTGRES_PORT", 5432),
        dbname=os.getenv("POSTGRES_DB", "chat-history"),
        user=os.getenv("POSTGRES_USER", "postgres"),
        password=os.getenv("POSTGRES_PASSWORD", "password")
    )

PostgresChatMessageHistory.create_tables(
    get_connection(),
    CHAT_HISTORY_TABLE # table name
)

In [ ]:
# NOTE: The variables in this notebook are based on the active session I have in my local machine.
# You may need to change the values if you want to run the code.

from langchain_postgres import PostgresChatMessageHistory
import psycopg
import os
from dotenv import load_dotenv

load_dotenv()

session_id = "c2b34f93-1dfa-49a0-9bc3-899c3465314e" # sample session_id

def get_connection():
    return psycopg.connect(
        host=os.getenv("POSTGRES_HOST", "localhost"),
        port=os.getenv("POSTGRES_PORT", 5432),
        dbname=os.getenv("POSTGRES_DB", "chat-history"),
        user=os.getenv("POSTGRES_USER", "postgres"),
        password=os.getenv("POSTGRES_PASSWORD", "password")
    )

chat_history = PostgresChatMessageHistory(
    "chat_history", # table name
    session_id, # session id
    sync_connection=get_connection()
)

chat_history.get_messages()

[HumanMessage(content='Describe what is an AI chatbot for beginners!', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content="An AI chatbot is a computer program that uses artificial intelligence (AI) to simulate human-like conversations with users through text or voice interactions. It's designed to answer questions, provide information, and perform tasks using natural language processing (NLP). Chatbots can be simple or complex, but their primary goal is to make interacting with technology feel more human-like and intuitive.", additional_kwargs={}, response_metadata={'done': True, 'model': 'llama3.2', 'created_at': '2025-06-08T10:51:05.590721898Z', 'eval_count': 75, 'model_name': 'llama3.2', 'done_reason': 'stop', 'eval_duration': 18058476023, 'load_duration': 16522514, 'total_duration': 19635443293, 'prompt_eval_count': 83, 'prompt_eval_duration': 1559748814}, id='run--d988a83f-48ad-4a30-a2b9-ba5b848b6b2a', usage_metadata={'input_tokens': 83, 'output_tokens': 75, 'tot

As you can see, the chat history is stored in the database. The next step is to be able to use the chat history to give more context to the LLM when generating a response.

For this, I chained the `PostgresChatMessageHistory` with the `ChatOllama` model and used the `RunnableWithMessageHistory` to give the chat history to the LLM.

See how we were able to give the chat history to the LLM in the following code:

In [ ]:
# NOTE: The variables in this notebook are based on the active session I have in my local machine.
# You may need to change the values if you want to run the code.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_postgres.chat_message_histories import PostgresChatMessageHistory

from app.core.prompts import get_session_title_prompt, chat_prompt

from dotenv import load_dotenv
import os

load_dotenv()

CHAT_HISTORY_TABLE = os.getenv("CHAT_HISTORY_TABLE", "chat_history")
session_id = "c2b34f93-1dfa-49a0-9bc3-899c3465314e" # sample session_id

llm = ChatOllama(model=OLLAMA_MODEL, base_url=OLLAMA_URL)

sync_connection = get_connection()

def get_session_title_chain():
    return get_session_title_prompt() | llm

def get_chat_chain():
    return chat_prompt() | llm

def get_chat_chain_with_history(chat_history_table, connection):
    return RunnableWithMessageHistory(
        get_chat_chain(),
        lambda session_id: PostgresChatMessageHistory(
            chat_history_table,
            session_id,
            sync_connection=connection
        ),
        input_messages_key="user_input",
        history_messages_key="history"
    )

# give it a try
llm_with_history = get_chat_chain_with_history(CHAT_HISTORY_TABLE, get_connection())

response = llm_with_history.invoke({"user_input": "What's my last message?"}, config={"configurable": {"session_id": session_id}})


See that the LLM was able to give the last message from the chat history:

In [22]:
print(response.content)

Your last message was: "Describe what is an AI chatbot for beginners!"


The runnable `RunnableWithMessageHistory` was used because it allows to achieve the core requirement to store chatbot memory with `langchain-postgres`. It's a wrapper that connects to the memory (Postgres), retrieves the chat history of a given session and passes it to the LLM, and it also automatically updates the memory with the new message and the response from the LLM. It simplifies the process of memory preservation, and, at the same time, it eases the session handling for the user.

In summary, the following are the steps we took to develop the basic chaining of the LLM:

1. We started by creating a simple chain that uses the `ChatOllama` model to generate a response.
2. We then added the `PostgresChatMessageHistory` to the chain to store the chat history.
3. We then integrated the LLM and the database through the `RunnableWithMessageHistory` wrapper.

In the next steps, we will be discussing the additional features that we added to the chatbot.

To allow the user to select from past sessions, we created a feature that allows user to see the previous sessions by listing the distinct session IDs in the database.

A sample code is shown below:

In [23]:
# NOTE: The variables in this notebook are based on the active session I have in my local machine.
# You may need to change the values if you want to run the code.

def list_sessions():
    cursor = get_connection().cursor()
    # get distinct sessions
    cursor.execute("""
        WITH latest_messages AS (
            SELECT session_id, MAX(created_at) as last_activity
            FROM chat_history
            GROUP BY session_id
        )
        SELECT session_id 
        FROM latest_messages 
        ORDER BY last_activity ASC
    """)
    return [row[0] for row in cursor.fetchall()]

list_sessions()

[UUID('9029ee99-c701-468a-820d-addb23ff1281'),
 UUID('c2b34f93-1dfa-49a0-9bc3-899c3465314e')]

At this point, we see that the user has created two sessions. But the problem here is that user won't be able to view the context of the session as the list displayed is just the session IDs.

To allow the user to view the context of the session, we created a feature that automatically generates a title for the session based on the last message. You may view the whole orchestration of the session title generation in `app/services/chat_sessions.py`.

For now, we will discuss the functions on a function-by-function basis.

For generating the session title, we used the following chain:

In [27]:
# NOTE: The variables in this notebook are based on the active session I have in my local machine.
# You may need to change the values if you want to run the code.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv

load_dotenv()

OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")

def get_session_title_prompt():
    return ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that creates concise, descriptive titles for chat sessions. Create a title that captures the main topic or purpose of the conversation. \
        Create a short title (3-5 words) for a chat session that starts with the message given. Do not include any other text in your response. \
        If the message is vague -- no context or just a greeting, return 'New Chat'"),
        ("user", "{message}")
    ])

# give it a try
llm = ChatOllama(model=OLLAMA_MODEL, base_url=OLLAMA_URL)

get_title_chain = get_session_title_prompt() | llm

response = get_title_chain.invoke({"message": "Describe what is an AI chatbot for beginners!"})

print(response.content)


"Introduction to AI Chatbots"


Next, I have added an additional instruction to not generate a title if the message is vague:

In [ ]:
vague_message = "Hello, how are you?"

response = get_title_chain.invoke({"message": vague_message})

print(response.content)

New Chat


This is to avoid the LLM from generating a title that is too vague, thus retaining the title as "New Chat". When the user has provided enough context already, an additional function was written so that the session is renamed using up to the last 4 questions and answers, to be discussed in the next section.

After generating the session title, it is stored in the database along with the session ID to make sure that the title matches the correct session. See below:

In [ ]:
# NOTE: The variables in this notebook are based on the active session I have in my local machine.
# You may need to change the values if you want to run the code.

SESSION_TITLES_TABLE = os.getenv("SESSION_TITLES_TABLE", "session_titles")

cursor = get_connection().cursor()
cursor.execute(f"SELECT session_id, title FROM {SESSION_TITLES_TABLE}")

result = cursor.fetchall()

for row in result:
    print(f"Session ID: {row[0]}, Title: {row[1]}")

Session ID: c2b34f93-1dfa-49a0-9bc3-899c3465314e, Title: "Introduction to AI Chatbots Basics"
Session ID: 9029ee99-c701-468a-820d-addb23ff1281, Title: The Meaning of Existence and Purpose


Now, we have the session titles in the database!

## 4. Frontend: Streamlit Chat UI

Streamlit provides a fast and intuitive way to build interactive UIs using pure Python. One of its biggest advantages is that it allows **direct integration with backend functions**, eliminating the need to build endpoints. This enables faster iteration and keeps the project focused on core functionality rather than API boilerplate.

By using Streamlit, we were able to rapidly prototype and test the chatbot interface while maintaining tight coupling with LangChain and the chat history logic.

**Key parts of the UI:**
- Session selection sidebar
- User input form using `st.chat_input()`
- Real-time response display using `st.write_stream()`


We divided the frontend into two components: the sidebar and the main chat interface.

The sidebar is used to select the session and create a new session. It also displays the chat history.

The main chat interface is used to display the chat history and the user input form. It also displays the response from the LLM.

A screenshot of the frontend is shown below:

![chat-history](public/images/chat-history.png)

Here is the streamlit code for the sidebar:

```python
# session management
if "sessions" not in st.session_state:
    st.session_state["sessions"] = [str(session_id) for session_id in list_sessions()]

# default session_id is None
if "session_id" not in st.session_state:
    st.session_state["session_id"] = None

st.set_page_config(
    page_title="Ollama Chatbot",
)

with st.sidebar:
        
    # create new session button
    st.button("New Chat", on_click=create_new_session, use_container_width=True)
    
    st.divider()
    
    st.header("Chat History")

    # list session history
    for session in st.session_state["sessions"]:
        session_title_col, delete_col = st.columns([4, 1])
        with session_title_col:
            session_title = get_session_title_sync(session) or f"New Chat" # get session title or default to "New Chat" if no title yet

            if session == st.session_state["session_id"]:
                st.button(
                    session_title,
                    key=f"session_{session}",
                    use_container_width=True,
                    type="primary" # emphasize the current session
                )
            else:
                if st.button(
                    session_title,
                    key=f"session_{session}",
                    use_container_width=True
                ):
                    st.session_state["session_id"] = session 
                    st.session_state["history"] = []
                    st.rerun()
        with delete_col: # delete session button
            if st.button("🗑️", key=f"delete_{session}", use_container_width=True):
                st.session_state["sessions"].remove(session)
                run_async(delete_session(session))
                st.rerun()
```

If the user wants a new session, they can click the "New Chat" button in the sidebar. This will create a new page with no session ID (see below):

![newchat](public/images/new-chat.png)

The session ID will not be generated until the user enters the first message. This is to avoid generating a session ID every time the user clicks the "New Chat" button.

Next, when a user enters a query, the following logic is executed:

1. If the session ID is not set, a new session is created and the session ID is set.
2. If the session ID is set, the query is sent to the LLM.
3. The response from the LLM is displayed in the chat interface.
4. The chat history is updated with the new message and the response from the LLM.
5. The session title is updated asynchronously with the new message.
6. The chat history is displayed in the chat interface.
7. The session title is displayed in the sidebar.

A sample code is shown below:

```python
# user input
user_input = st.chat_input("Type your message here...", key="user_input")
if user_input and user_input != "":
    with st.chat_message("user"):
        st.markdown(f"{user_input}")
    with st.chat_message("assistant"):
        if session_id is None:  # if this is a new chat, create a new session
            session_id = str(uuid.uuid4())
            st.session_state["session_id"] = session_id
            st.session_state["sessions"] = [session_id] + st.session_state["sessions"] # add new session to the top of the list
            thread_pool.submit(run_async, create_session_title(session_id, user_input)) # create session title asynchronously
            st.write_stream(get_response_stream(session_id, user_input))
            st.rerun()
        else:
            st.write_stream(get_response_stream(session_id, user_input))
            if get_session_title_sync(session_id) == "New Chat": # rename session after getting more information from the session
                message_history = get_chat_history(session_id)
                messages = "\n".join(f"{msg['type']}: {msg['content']}" for msg in message_history[-4:]) 
                new_title = run_async(update_session_title(session_id, messages))
                if new_title:
                    st.session_state["session_title"] = new_title
            st.rerun()  # refresh the UI to update sidebar and title
```

As discussed in the previous section, when the first message is vague, the session title is set to "New Chat". A logic was added in the streamlit app to check if the session title is "New Chat" and if so, update the session title with the last 4 messages.

Lastly, a streaming response `st.write_stream()` was used to display the response from the LLM in real-time. This will allow the user to see the response as it is being generated, rather than waiting for the entire response to be generated before displaying it, allowing for a more natural conversation experience.

## 5. Running the Chatbot

We have created a containerized application that can be run locally.

To run the application, you need to have the following:

- Docker
- Docker Compose

To run the application, run the following command:

```bash
docker compose up --build
```

After the build is complete, you can access the application at `http://localhost:8501`.

For more information on how to run the application, please refer to the [README](README.md).

## 6. Tests

We wrote unit tests for:
- Chat streaming logic
- Chat history retrieval
- Database session listing

For the integration tests, we used `pytest` with mocking to isolate components.

Run the tests with the following command when you are running via docker compose:

```bash
docker-compose exec app pytest
```

If running locally, you can run the tests with the following command:

```bash
pytest
```

View [README](README.md) for more information on how to run the tests.

## 7. Challenges faced

### Parallel Execution of Title Creation and Response Generation

One of the key challenges I encountered during development was managing the execution timing of two separate backend tasks:
- Generating the session title
- Streaming the chatbot response

Initially, these were executed sequentially. This led to noticeable delays and a less responsive UI. To solve this, I introduced `asyncio` to run both tasks in parallel, since they are independent of each other. This reduced the overall wait time and significantly improved the user experience. This idea has also led me to update other operations, like delete a session, to be asynchronous as well to further reduce the frontend latency.

### Containerizing the Application

Another challenge was containerizing the whole application (App, Postgres, and Ollama). While I had limited experience with Docker previously, I decided to adopt it for this project to make the app:
- More portable
- Easier to run 

Through this, I gained more experience with Dockerfile, docker-compose, and handling service-to-service networking (e.g., database and model container communication). 

## Conclusion

This project demonstrates the design and development of a local, self-contained LLM chatbot using modern tooling:
- **Ollama** for efficient local LLM inference
- **LangChain** for chaining model logic and integrating memory
- **PostgreSQL** for persistent session memory
- **Streamlit** for an interactive and minimal frontend
- **Docker** for full application portability and environment consistency

Throughout the process, I focused on modularity, performance, and user experience. Asynchronous logic was introduced to parallelize tasks like response generation and session title creation. Containerization was adopted to improve deployability and reproducibility, even though it initially posed a learning curve.